In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import joblib

In [2]:
# 1. Download BTC Data
df = yf.download('BTC-USD', start='2014-01-01', end='2025-06-01')
df = df[['Close', 'Volume']]

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [3]:
# 2. Add Technical Indicators
df['MA20'] = df['Close'].rolling(window=20).mean()
df['MA50'] = df['Close'].rolling(window=50).mean()
df['Returns'] = df['Close'].pct_change()
df['PriceDiff'] = df['Close'].diff()

In [4]:
# RSI Calculation
delta = df['Close'].diff()
gain = (delta.where(delta > 0, 0)).rolling(14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
rs = gain / loss
df['RSI'] = 100 - (100 / (1 + rs))

In [5]:
# MACD
ema12 = df['Close'].ewm(span=12, adjust=False).mean()
ema26 = df['Close'].ewm(span=26, adjust=False).mean()
df['MACD'] = ema12 - ema26
df['Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()

df.dropna(inplace=True)

In [6]:
# 3. Feature Scaling
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df)

In [7]:
# Save the scaler
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [8]:
# 4. Sequence Preparation
X, y = [], []
sequence_length = 60

In [9]:
for i in range(sequence_length, len(scaled_data)):
    X.append(scaled_data[i-sequence_length:i])
    y.append(scaled_data[i, 0])  # Predict 'Close'

X, y = np.array(X), np.array(y)

In [10]:
# 5. Improved LSTM Model
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(X.shape[1], X.shape[2])),
    Dropout(0.3),
    LSTM(64, return_sequences=False),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1)
])

C:\Users\Prasanna dindi\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [11]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [13]:
# 6. Training with Callbacks
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint('best_btc_lstm_model.keras', save_best_only=True)
]

model.fit(X, y, epochs=50, batch_size=32, callbacks=callbacks)

# Save final model
model.save('btc_lstm_model.keras')

Epoch 1/50
119/119 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - loss: 0.0101
Epoch 2/50
  5/119 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - loss: 0.0028

C:\Users\Prasanna dindi\AppData\Roaming\Python\Python312\site-packages\keras\src\callbacks\early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
  current = self.get_monitor_value(logs)
C:\Users\Prasanna dindi\AppData\Roaming\Python\Python312\site-packages\keras\src\callbacks\model_checkpoint.py:206: UserWarning: Can save best model only with val_loss available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


119/119 ━━━━━━━━━━━━━━━━━━━━ 4s 37ms/step - loss: 0.0019
Epoch 3/50
119/119 ━━━━━━━━━━━━━━━━━━━━ 4s 37ms/step - loss: 0.0014
Epoch 4/50
119/119 ━━━━━━━━━━━━━━━━━━━━ 4s 37ms/step - loss: 0.0012
Epoch 5/50
119/119 ━━━━━━━━━━━━━━━━━━━━ 4s 37ms/step - loss: 0.0012
Epoch 6/50
119/119 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - loss: 0.0011
Epoch 7/50
119/119 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - loss: 0.0010
Epoch 8/50
119/119 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - loss: 0.0011
Epoch 9/50
119/119 ━━━━━━━━━━━━━━━━━━━━ 5s 38ms/step - loss: 8.8835e-04
Epoch 10/50
119/119 ━━━━━━━━━━━━━━━━━━━━ 5s 39ms/step - loss: 9.1996e-04
Epoch 11/50
119/119 ━━━━━━━━━━━━━━━━━━━━ 4s 37ms/step - loss: 9.9800e-04
Epoch 12/50
119/119 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - loss: 6.9180e-04
Epoch 13/50
119/119 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - loss: 7.7104e-04
Epoch 14/50
119/119 ━━━━━━━━━━━━━━━━━━━━ 5s 39ms/step - loss: 8.6190e-04
Epoch 15/50
119/119 ━━━━━━━━━━━━━━━━━━━━ 5s 39ms/step - loss: 8.3045e-04
Epoch 16/50
119/119 ━━━━━